In [7]:
import hysys_python.hysys_object_persistence as hop
import hysys_gsa_util as hgu
import numpy as np
import time as ti
from IPython.display import clear_output
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [8]:
filepath = r"C:\Users\nt320\OneDrive - Imperial College London\Niki GSA bayesian optimisation\Submission 2"
file = filepath + r"\i-dme-complete-gsa-equil.hsc"

# Creating and connecting to the hysys flowsheet and solve
try:
    sim = hop.hysys_connection(file, active=1)
except:
    sim = hop.hysys_connection(file, active=0)

fsheet = sim.Flowsheet
solver = sim.Solver


In [9]:
## BOUNDS
# electricity prices - 2018-2023 https://tradingeconomics.com/united-kingdom/electricity-price
elec_nom = 89.2 # GBP/MWh - 02 Jan 2024
elec_nom *= 0.804354 # USD/MWh
elec_nom /= 1000 # USD/kWh

# For the nominal/range of the MeOH reactor size from van Dal 2013
ref_co2_flow = 88e3 # kg/h CO2
co2_flow = 28333.3620500565 # kg/h in simulation
co2_ratio = co2_flow/ref_co2_flow
ref_cat_mass = 44500 # kg catalyst
cat_mass = ref_cat_mass*co2_ratio # kg catalyst in simulation
void = 0.5
density = 1775
meoh_nominal_vol = cat_mass * (1/density) * (1/(1-void)) # m3

bounds = [[2.4*0.9, 3.6*1.1],       # h2 ratio - +/-20% of stoich
        [5000*0.9, 10000*1.1],    # meoh pressure - van-dal2013
        [210*0.9, 270*1.1],       # meoh feed temp - van-dal2013
        [0, 1],           # adiabatic/isothermal meoh
        [0.94, 0.995],      # recycle ratio
        [0.8*meoh_nominal_vol*0.9, 1.2*meoh_nominal_vol*1.1], # meoh reactor volume - van-dal2013 +/- 20%
        [250*0.9, 300*1.1],       # dme feed temperature - peinado2020    
        [1000*0.9, 2000*1.1],     # dme reaction pressure - peinado2020
        [0,1],            # feed vapour fraction meoh column
        [0,1],            # feed vapour fraction dme-meoh column
        [57*0.8*0.9, 57*1.2*1.1], # trays col 1 +/- 20% of nominal
        [17*0.8*0.9, 17*1.2*1.1], # trays col 2 +/- 20% of nominal
        [44/57*0.8, 44/57*1.2], # relative feed location col 1 +/- 20% of nominal
        [10/17*0.8, 10/17*1.2]  # relative feed location col 2 +/- 20% of nominal
        ]


In [10]:
iter =0
average_solve_time = 0
failures = 0
for i in range(iter):
    # Preamble
    print(f"Iteration {i+1} of {iter}, {failures} failures")
    print(f"Average solve time = {average_solve_time}s")
    predicted_seconds = float(average_solve_time) * (iter - i)
    predicted_hours = int(predicted_seconds // 3600)
    predicted_minutes = int((predicted_seconds % 3600) // 60)
    predicted_seconds = int(predicted_seconds % 60)
    predicted = f"{predicted_hours}h {predicted_minutes}m {predicted_seconds}s"
    print(f"Predicted time remaining = {predicted}\n")

    # Generating random inputs - SWAP WITH DESIRED GSA INPUTS
    inputs = hgu.random_inputs(nominal=False)

    # Running the flowsheet
    tic = ti.perf_counter()
    sim.Visible = False # hiding the simulation improves solve time
    out = hgu.solve_calc_flowsheet(fsheet, solver, inputs)

    # Printing the results
    if type(out) != type(None):
        print('DME flowrate         = {:.2f} kg/s'.format(out[0]))
        print('Carbon efficiency    = {:.1f}%'.format(out[1]))
        print('Energy efficiency    = {:.1f}%'.format(out[5]))
        print('CAPEX                = {:.0f} M USD'.format(out[2]/1e6))
        print('OPEX                 = {:.0f} M USD'.format(out[3]/1e6))
        print('LCP                  = {:.2f} USD/kg'.format(out[4]))
    
    if type(out[0]) == type(None):
        failures += 1

    # Performing time calculations
    toc = ti.perf_counter()
    formatted_elapsed_time = "{:.2f}".format(toc-tic)
    print(f"Solved script without unhiding in {formatted_elapsed_time}s\n\n")
    average_solve_time = (float(average_solve_time) * i + (toc-tic)) / (i+1)
    average_solve_time = "{:.2f}".format(average_solve_time)
    clear_output(wait=True)

sim.Visible = True # unhide the simulation case


In [11]:
## Test GSA code
from SALib.sample import saltelli
from SALib.analyze import sobol
from IPython.display import clear_output
import pandas as pd

In [ ]:
import numpy as np
from datetime import datetime

# Set the seed
np.random.seed(42)

from SALib.sample import sobol_sequence

problem = {
    'num_vars': 14,
    'names': ['h2_ratio','Pmeoh', 'Tmeoh','reactor_mode', 'recycle_ratio', 'Vmeoh', 'Tdme', 'Pdme', 'xmeoh', 'xdme', 'n_trays1', 'n_trays2', 'feed_loc1', 'feed_loc2'],
    'bounds': bounds,
    'dists': ['unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif', 'unif']
}

param_values = saltelli.sample(problem, 175, calc_second_order=True)
# Generate the initial set of Sobol samples
print(len(param_values))


# Create a DataFrame with the parameter values
param_df = pd.DataFrame(param_values, columns=problem['names'])

# Save the DataFrame to a CSV file
param_df.to_csv('inputs.csv', index=False)

5250


C:\Users\nt320\AppData\Local\Temp\ipykernel_8500\441884004.py:16: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 175, calc_second_order=True)
C:\Users\nt320\AppData\Local\Programs\Python\Python311\Lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (175) is equal to `2^n`.
        
  warnings.warn(msg)


In [ ]:

data = pd.read_csv('inputs.csv')

dataset = data

#dataset to numpy array
param_values = dataset.to_numpy()


In [ ]:
columns = ['DME flowrate', 'Carbon efficiency', 'CAPEX', 'OPEX', 'LCP', 'Energy efficiency']
df = pd.DataFrame(columns=columns)

iter = len(param_values)
average_solve_time = 0
failures = 0
sim.Visible = False # hiding the simulation improves solve time

for i in range(len(param_values)):
    # Preamble
    print(f"Iteration {i+1} of {iter}, {failures} failures")
    print(f"Average solve time = {average_solve_time}s")
    predicted_seconds = float(average_solve_time) * (iter - i)
    predicted_hours = int(predicted_seconds // 3600)
    predicted_minutes = int((predicted_seconds % 3600) // 60)
    predicted_seconds = int(predicted_seconds % 60)
    predicted = f"{predicted_hours}h {predicted_minutes}m {predicted_seconds}s"
    print(f"Predicted time remaining = {predicted}\n")


    # Running the flowsheet
    tic = ti.perf_counter()
    try:
        out = hgu.solve_calc_flowsheet(fsheet, solver, param_values[i])
    except:
        out = [None, None, None, None, None, None]

    # Create a temporary DataFrame for the current iteration
    temp_df = pd.DataFrame([out], columns=columns)

    # Append the temporary DataFrame to the main DataFrame
    df = pd.concat([df, temp_df], ignore_index=True)
   
    if type(out[0]) == type(None):
        failures += 1

            # Save DataFrame periodically (e.g., every 10 iterations)
    if i % 100 == 0:
        df.to_csv('saved_dataframe.csv')

    # Performing time calculations
    toc = ti.perf_counter()
    formatted_elapsed_time = "{:.2f}".format(toc-tic)
    print(f"Solved script without unhiding in {formatted_elapsed_time}s\n\n")
    average_solve_time = (float(average_solve_time) * i + (toc-tic)) / (i+1)
    average_solve_time = "{:.2f}".format(average_solve_time)
    clear_output(wait=True)

    #sim.Visible = True # unhide the simulation case
    


df.to_csv('outputs.csv')

Iteration 5250 of 5250, 34 failures
Average solve time = 15.36s
Predicted time remaining = 0h 0m 15s

Set inputs in 0.14s
Initial solve up to Reactor100 in 0.21s
Solved Reactor100 in 0.29s
Solved MIX-101_meoh_rcy in 0.11s
Solved K-103 in 0.11s
Solved Vessel 101 in 0.12s
Solved Twp101 in 0.34s
Solved V-100 in 0.26s
Solved CRV-100 in 0.14s
Solved Twb102 in 0.42s
Solved MIX-100_met_rcy in 0.41s
*** COMPLETE ***
Solved flowsheet in 2.45s

Sized DME reactor in 2.58s
Solved script without unhiding in 6.23s


